In [ ]:
pip install opencv-python

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

In [ ]:
from pyspark.sql import SparkSession
import cv2
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType, StringType, StructType, StructField
import os
import pandas as pd

In [ ]:
spark = SparkSession.builder \
    .appName("ResizeImages") \
    .getOrCreate()

In [ ]:
input_directory = '/dbfs/mnt/projectteam5/New_Dataset/'

In [ ]:
output_directory = '/dbfs/mnt/projectteam5/pyspark_output/'

In [ ]:
def resize_image(image_path, width=250, height=250):
    img = cv2.imread(image_path)
    resized_img = cv2.resize(img, (width, height))
    retval, buffer = cv2.imencode('.jpg', resized_img)
    resized_image_data = buffer.tobytes()
    return resized_image_data

In [ ]:
resize_image_udf = udf(resize_image, BinaryType())
# resize_image_udf = udf(resize_image, StringType())

In [ ]:
image_paths = [os.path.join(input_directory, file) for file in os.listdir(input_directory)]

In [ ]:
schema = StructType([StructField("image_path", StringType(), nullable=True)])

In [ ]:
image_paths_df = spark.createDataFrame([(path,) for path in image_paths], schema)

In [ ]:
resized_images_df = image_paths_df.withColumn("resized_image", resize_image_udf(image_paths_df["image_path"]))

In [ ]:
pandas_df = resized_images_df.toPandas()

In [ ]:
for index, row in pandas_df.iterrows():
    image_data = np.frombuffer(row['resized_image'], dtype=np.uint8)
    img = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
    cv2.imwrite(os.path.join(output_directory, f"resized_image_{index}.jpg"), img)

In [ ]:
resized_images_df

In [ ]:
schema 